In [1]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, Dataset, DataLoader

class MissingEntryDataset(Dataset):
    def __init__(self, data: torch.tensor, min_missing: float, max_missing: float):
        self.data = data

        n, d = data.shape
        self.min_missing = min_missing
        self.max_missing = max_missing
        
    def __getitem__(self, idx):
        sample = self.data[idx]
        
        prob = np.random.uniform(self.min_missing, self.max_missing)
        mask = np.random.binomial(1, prob, sample.shape)
        mask = torch.tensor(mask).bool()

        return sample, mask
    
    def __len__(self):
        return len(self.data)

In [2]:
import pandas as pd
df = pd.read_csv("TCPA_data_sel.csv")

# select the 189 real valued columns only
X = df.iloc[:, 2:].values.astype("float32")
Xs = (X - X.mean(axis=0)) / X.var(axis=0)

# split train/test
n = len(Xs)
idx = np.arange(len(Xs))
ncut = int(n * 0.8)
Xtrain = MissingEntryDataset(torch.tensor(Xs[idx[:ncut]]), 0.1, 0.4)
Xtest = MissingEntryDataset(torch.tensor(Xs[idx[ncut:]]), 0.1, 0.4)
len(Xtrain), len(Xtest)

(3784, 946)

In [12]:
import torch
from torch import nn
import torch.nn.functional as F

class LinSkip(nn.Module):
    def __init__(self, ins):
        super(LinSkip, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(ins, ins),
            nn.LeakyReLU(),
            nn.BatchNorm1d(ins))
        
    def forward(self, x):
        return x + self.layers(x)

class VAE(nn.Module):
    def __init__(self, ins=189, hidden=512, latent=64, variational=True):
        super(VAE, self).__init__()
        self.variational = variational
        
        self.enc = nn.Sequential(nn.Linear(ins, hidden),
                                 nn.LeakyReLU(),
                                 nn.BatchNorm1d(hidden),
                                 LinSkip(hidden),
                                 LinSkip(hidden),
                                 LinSkip(hidden),
                                 LinSkip(hidden))
        
        self.mean = nn.Linear(hidden, latent)
        self.log_variance = nn.Linear(hidden, latent)
        
        self.dec = nn.Sequential(nn.Linear(latent, hidden),
                                 nn.LeakyReLU(),
                                 nn.BatchNorm1d(hidden),
                                 LinSkip(hidden),
                                 LinSkip(hidden),
                                 LinSkip(hidden),
                                 LinSkip(hidden),
                                 nn.Linear(hidden, ins))

    def sample(self, mean, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x, mask):
        x[mask] = torch.randn(mask.shape)[mask]
        x = self.enc(x)
        mu = self.mean(x)
        log_var = self.log_variance(x)
        z = self.sample(mu, log_var) if self.variational else mu
        x = self.dec(z)
        return x, mu, log_var
    
    def gibbs(self, x0, mask):
        # create copy
        xn = x0[:]
        # iterativly predict
        xns = []
        for _ in range(20):
            # reconstruction step
            xn,_,_ = self.forward(xn, mask)
            xns.append(xn)
            # reset observed values
            xn[~mask] = x0[~mask]
        xn = torch.mean(torch.stack(xns, dim=-1), dim=-1)
        return xn
    
vae = VAE(hidden=512, latent=128, variational=True)
print(vae)
assert vae(torch.tensor(np.random.randn(20, 189).astype("float32")),
           torch.tensor(np.random.randn(20, 189).astype("bool")))

VAE(
  (enc): Sequential(
    (0): Linear(in_features=189, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LinSkip(
      (layers): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
        (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (4): LinSkip(
      (layers): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
        (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (5): LinSkip(
      (layers): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
        (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [13]:
import torch.optim as optim
optimizer = optim.Adam(vae.parameters(), lr=0.0001)

def train(epoch):
    sample, mask = Xtrain[:]
    reconstruction = sample[:]
    for n in range(10):
        vae.train()
        optimizer.zero_grad()
        reconstruction, mu, log_var = vae(reconstruction, mask)
        mse = F.mse_loss(reconstruction, sample, reduction="sum")
        kl = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        loss = mse + 1 * kl  # set to 1 for variational regularization (centered gaussian)
        loss.backward()
        optimizer.step()
        print(f"Train Epoch {epoch}.{n}", 
              f"var loss {loss.item()}", 
              f"reconstruction mse {mse.item()}",
              f"imputation mse {F.mse_loss(reconstruction[mask], sample[mask], reduction='mean')}")
        reconstruction = reconstruction.detach() # drop previous computation graph before going into next iter
        reconstruction[~mask] = sample[~mask]
    
def test():
    vae.eval()
    with torch.no_grad():
        sample, mask = Xtest[:]
        reconstruction = vae.gibbs(sample, mask)   
        test_loss = F.mse_loss(reconstruction[mask], sample[mask], reduction="mean").item()
    print('====> Test imputation mse: {:.8f}'.format(test_loss))

In [14]:
test()
test()
test()
for epoch in range(1, 10000000):
    train(epoch)
    if epoch%10==0:
        test()
        test()
        test()

====> Test imputation mse: 1.05301106
====> Test imputation mse: 1.04914165
====> Test imputation mse: 1.04920268
Train Epoch 1.0 var loss 299094671360.0 reconstruction mse 4379207.5 imputation mse 4.079547882080078
Train Epoch 1.1 var loss 714089664.0 reconstruction mse 4407217.0 imputation mse 4.034031391143799
Train Epoch 1.2 var loss 99893784.0 reconstruction mse 4168317.0 imputation mse 3.8297300338745117
Train Epoch 1.3 var loss 131044880.0 reconstruction mse 3920679.0 imputation mse 3.415123701095581
Train Epoch 1.4 var loss 83495440.0 reconstruction mse 3682715.25 imputation mse 3.0968034267425537
Train Epoch 1.5 var loss 116868720.0 reconstruction mse 3529519.25 imputation mse 2.828763008117676
Train Epoch 1.6 var loss 42980480.0 reconstruction mse 3334655.5 imputation mse 2.6962342262268066
Train Epoch 1.7 var loss 59920172.0 reconstruction mse 3261506.75 imputation mse 2.613262414932251
Train Epoch 1.8 var loss 30513344.0 reconstruction mse 3163430.25 imputation mse 2.408326

Train Epoch 9.2 var loss 1765222.375 reconstruction mse 1193182.5 imputation mse 1.6415743827819824
Train Epoch 9.3 var loss 1726023.0 reconstruction mse 1187348.625 imputation mse 1.6351244449615479
Train Epoch 9.4 var loss 1665059.25 reconstruction mse 1171981.125 imputation mse 1.6187598705291748
Train Epoch 9.5 var loss 1745665.625 reconstruction mse 1207826.0 imputation mse 1.6662957668304443
Train Epoch 9.6 var loss 1690887.125 reconstruction mse 1151322.875 imputation mse 1.5748322010040283
Train Epoch 9.7 var loss 1783963.25 reconstruction mse 1174808.375 imputation mse 1.6160300970077515
Train Epoch 9.8 var loss 1709975.0 reconstruction mse 1179843.875 imputation mse 1.6269973516464233
Train Epoch 9.9 var loss 1688816.375 reconstruction mse 1186778.25 imputation mse 1.6351354122161865
Train Epoch 10.0 var loss 1620979.5 reconstruction mse 1153319.5 imputation mse 1.523240327835083
Train Epoch 10.1 var loss 1622570.75 reconstruction mse 1173457.875 imputation mse 1.647442340850

Train Epoch 17.3 var loss 1270239.0 reconstruction mse 953509.375 imputation mse 1.382903814315796
Train Epoch 17.4 var loss 1268878.0 reconstruction mse 951789.4375 imputation mse 1.3806519508361816
Train Epoch 17.5 var loss 1263690.0 reconstruction mse 949112.0 imputation mse 1.375594139099121
Train Epoch 17.6 var loss 1260072.0 reconstruction mse 946791.375 imputation mse 1.3711674213409424
Train Epoch 17.7 var loss 1259892.25 reconstruction mse 948693.0625 imputation mse 1.376996636390686
Train Epoch 17.8 var loss 1259267.0 reconstruction mse 946614.3125 imputation mse 1.3749547004699707
Train Epoch 17.9 var loss 1251556.5 reconstruction mse 941774.8125 imputation mse 1.3638010025024414
Train Epoch 18.0 var loss 1224837.75 reconstruction mse 916757.1875 imputation mse 1.2830249071121216
Train Epoch 18.1 var loss 1241701.75 reconstruction mse 934485.6875 imputation mse 1.3778527975082397
Train Epoch 18.2 var loss 1234922.75 reconstruction mse 928998.25 imputation mse 1.3654075860977

Train Epoch 25.4 var loss 1100459.75 reconstruction mse 858152.0 imputation mse 1.2596590518951416
Train Epoch 25.5 var loss 1095834.125 reconstruction mse 855635.75 imputation mse 1.2474653720855713
Train Epoch 25.6 var loss 1095344.75 reconstruction mse 856931.3125 imputation mse 1.257463812828064
Train Epoch 25.7 var loss 1094055.75 reconstruction mse 856527.0 imputation mse 1.2540138959884644
Train Epoch 25.8 var loss 1091700.375 reconstruction mse 854897.9375 imputation mse 1.2521021366119385
Train Epoch 25.9 var loss 1090177.75 reconstruction mse 854506.75 imputation mse 1.248626708984375
Train Epoch 26.0 var loss 1073779.625 reconstruction mse 838979.5625 imputation mse 1.1688563823699951
Train Epoch 26.1 var loss 1087840.5 reconstruction mse 853773.25 imputation mse 1.2367359399795532
Train Epoch 26.2 var loss 1087266.5 reconstruction mse 853510.875 imputation mse 1.23895263671875
Train Epoch 26.3 var loss 1087103.5 reconstruction mse 853534.125 imputation mse 1.235742092132568

Train Epoch 33.4 var loss 996675.25 reconstruction mse 804703.125 imputation mse 1.1753654479980469
Train Epoch 33.5 var loss 996430.8125 reconstruction mse 804996.0 imputation mse 1.172358751296997
Train Epoch 33.6 var loss 994830.9375 reconstruction mse 804130.75 imputation mse 1.1704347133636475
Train Epoch 33.7 var loss 993231.25 reconstruction mse 803453.4375 imputation mse 1.1670758724212646
Train Epoch 33.8 var loss 991984.5 reconstruction mse 802625.125 imputation mse 1.1730494499206543
Train Epoch 33.9 var loss 990771.9375 reconstruction mse 801525.8125 imputation mse 1.1667917966842651
Train Epoch 34.0 var loss 983571.375 reconstruction mse 794810.4375 imputation mse 1.1164820194244385
Train Epoch 34.1 var loss 998865.6875 reconstruction mse 811655.1875 imputation mse 1.1773539781570435
Train Epoch 34.2 var loss 998869.375 reconstruction mse 811644.5 imputation mse 1.1764949560165405
Train Epoch 34.3 var loss 996570.1875 reconstruction mse 810733.0625 imputation mse 1.1770949

...

Train Epoch 138.3 var loss 781850.6875 reconstruction mse 725723.375 imputation mse 1.029941201210022
Train Epoch 138.4 var loss 781432.0 reconstruction mse 725860.375 imputation mse 1.0318509340286255
Train Epoch 138.5 var loss 781573.9375 reconstruction mse 725911.9375 imputation mse 1.0300980806350708
Train Epoch 138.6 var loss 781548.875 reconstruction mse 725735.0625 imputation mse 1.0294854640960693
Train Epoch 138.7 var loss 781347.1875 reconstruction mse 725762.3125 imputation mse 1.0305360555648804
Train Epoch 138.8 var loss 781755.125 reconstruction mse 725668.5 imputation mse 1.0305265188217163
Train Epoch 138.9 var loss 781332.25 reconstruction mse 725748.0 imputation mse 1.0295084714889526
Train Epoch 139.0 var loss 779734.125 reconstruction mse 724613.9375 imputation mse 1.0109978914260864
Train Epoch 139.1 var loss 780961.5 reconstruction mse 725915.625 imputation mse 1.0281729698181152
Train Epoch 139.2 var loss 781187.1875 reconstruction mse 725757.0 imputation mse 1.0

Train Epoch 146.3 var loss 775127.125 reconstruction mse 720451.1875 imputation mse 1.023151159286499
Train Epoch 146.4 var loss 775113.25 reconstruction mse 720568.625 imputation mse 1.0216355323791504
Train Epoch 146.5 var loss 775191.6875 reconstruction mse 720362.125 imputation mse 1.0219557285308838
Train Epoch 146.6 var loss 774527.75 reconstruction mse 720217.0 imputation mse 1.024253249168396
Train Epoch 146.7 var loss 774808.125 reconstruction mse 720431.6875 imputation mse 1.02267587184906
Train Epoch 146.8 var loss 775075.625 reconstruction mse 720657.9375 imputation mse 1.0219347476959229
Train Epoch 146.9 var loss 774717.125 reconstruction mse 720384.8125 imputation mse 1.0224870443344116
Train Epoch 147.0 var loss 774841.625 reconstruction mse 721263.375 imputation mse 1.011479377746582
Train Epoch 147.1 var loss 779432.5625 reconstruction mse 725304.6875 imputation mse 1.0294255018234253
Train Epoch 147.2 var loss 779363.6875 reconstruction mse 725773.625 imputation mse 

Train Epoch 154.3 var loss 776693.125 reconstruction mse 725105.4375 imputation mse 1.017844796180725
Train Epoch 154.4 var loss 778820.375 reconstruction mse 725802.9375 imputation mse 1.019150733947754
Train Epoch 154.5 var loss 777468.75 reconstruction mse 725078.3125 imputation mse 1.0183788537979126
Train Epoch 154.6 var loss 777458.0625 reconstruction mse 725325.375 imputation mse 1.0181949138641357
Train Epoch 154.7 var loss 776297.75 reconstruction mse 724798.9375 imputation mse 1.0162055492401123
Train Epoch 154.8 var loss 776783.9375 reconstruction mse 725022.375 imputation mse 1.0184924602508545
Train Epoch 154.9 var loss 777074.625 reconstruction mse 724846.5625 imputation mse 1.0175212621688843
Train Epoch 155.0 var loss 772790.8125 reconstruction mse 720835.1875 imputation mse 1.0084985494613647
Train Epoch 155.1 var loss 773723.8125 reconstruction mse 722045.6875 imputation mse 1.0228707790374756
Train Epoch 155.2 var loss 775632.875 reconstruction mse 722732.3125 imputa

Train Epoch 162.3 var loss 775519.375 reconstruction mse 725075.875 imputation mse 1.026511788368225
Train Epoch 162.4 var loss 776339.4375 reconstruction mse 725022.1875 imputation mse 1.0271164178848267
Train Epoch 162.5 var loss 776277.25 reconstruction mse 724935.5 imputation mse 1.0257434844970703
Train Epoch 162.6 var loss 776165.625 reconstruction mse 724626.1875 imputation mse 1.026313066482544
Train Epoch 162.7 var loss 775544.125 reconstruction mse 724787.625 imputation mse 1.0262082815170288
Train Epoch 162.8 var loss 775678.9375 reconstruction mse 724593.875 imputation mse 1.0253863334655762
Train Epoch 162.9 var loss 775410.625 reconstruction mse 724673.1875 imputation mse 1.0258628129959106
Train Epoch 163.0 var loss 771128.4375 reconstruction mse 720502.25 imputation mse 1.0049585103988647
Train Epoch 163.1 var loss 772940.0625 reconstruction mse 722084.875 imputation mse 1.01921546459198
Train Epoch 163.2 var loss 772843.875 reconstruction mse 722081.125 imputation mse 

KeyboardInterrupt: 